<a href="https://colab.research.google.com/github/azaleakamellia/kemarau/blob/main/Extracting%20monthly%20CHIRPS%20data%20as%20CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import the library, authenticate and initialize the Google Project you want to use for this project.

In [ ]:
import ee

ee.Authenticate()
ee.Initialize(project='ee-azaleakamellia')

##1️⃣ Load the CHIRPS Pentad ImageCollection

In [ ]:
chirps = ee.ImageCollection('UCSB-CHG/CHIRPS/PENTAD')

##2️⃣ Load the shapefile with coordinate points (replace with your asset ID)

In [ ]:
points = ee.FeatureCollection('projects/ee-azaleakamellia/assets/insitu_coords')

##3️⃣ Define the time range (1981 to 2024)

In [ ]:
startDate = ee.Date.fromYMD(1981, 1, 1)
endDate = ee.Date.fromYMD(2024, 12, 31)

##4️⃣Filter the CHIRPS data collection for the time range we want

In [ ]:
collection = chirps.select('precipitation').filterDate(startDate, endDate)

##5️⃣ Function to calculate monthly total precipitation

In [ ]:
def calculateMonthlySum(year, month):
    start = ee.Date.fromYMD(year, month, 1)
    end = start.advance(1, 'month')
    monthlyCollection = collection.filterDate(start, end)
    monthlySum = monthlyCollection.sum()  # Sum of images in the month
    return monthlySum.set('year', year).set('month', month)

##6️⃣ Create a list of years and months

In [ ]:
years = ee.List.sequence(1981, 2024)
months = ee.List.sequence(1, 12)

# Use a nested loop to calculate monthly images
monthlyImages = years.map(
    lambda y: months.map(
        lambda m: calculateMonthlySum(ee.Number(y), ee.Number(m))
    )
).flatten()

# Convert to an ImageCollection
monthlyImageCollection = ee.ImageCollection(monthlyImages)

##7️⃣ Add coordinates to the feature collection points & Extract data for points





In [ ]:
def addCoordinates(feature):
    coords = feature.geometry().coordinates()
    return feature.set('longitude', coords.get(0)).set('latitude', coords.get(1))

points = points.map(addCoordinates)  # Add coordinates to the points FeatureCollection

def extractData(image):
    # Sample the image at the points
    sampledPoints = image.sampleRegions(
        collection=points,  # Use the points with coordinates
        properties=['longitude', 'latitude'],  # Include coordinates in the output
        scale=5000,
        geometries=True
    )
    # Add year and month metadata to each sampled point
    return sampledPoints.map(
        lambda f: f.set({
            'year': image.get('year'),
            'month': image.get('month')
        })
    )

# Apply the extraction to each monthly image
extractedData = monthlyImageCollection.map(extractData).flatten()

##8️⃣ Export the data as a CSV

In [ ]:
task = ee.batch.Export.table.toDrive(
    collection=extractedData,
    description='CHIRPS_Monthly_Precipitation_upgraded',
    folder='earthengine',
    fileNamePrefix='CHIRPS_monthly_precipitation_upgraded',
    fileFormat='CSV',
    selectors=['year', 'month', 'longitude', 'latitude', 'precipitation']
)

task.start()